In [21]:
import math
import os
from methods import Segmenter, TransformerModel, LSTM, CRNN, UNet,CCRNN, UNet2, PatchTST
from matplotlib import pyplot as plt
import torch
from utilities import printc, seed
from utils_loader import get_dataloaders
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from utils_metrics import mean_iou
from torch.nn import functional as F

config = {
        'batch_size': 128,
        'epochs':200,
        'fsl': False,
        'model': 'unet',
        'seed': 73054772,
        'dataset': 'physiq',
        'window_size': 50,
        'step_size': 50,
    }
seed(config['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
from sklearn.model_selection import train_test_split
from utilities import sliding_windows
from utils_dataset import NormalDataset
from torch.utils.data import Dataset, DataLoader

from utils_loader import _load

def sec_sliding(samples, labels, step=50):
    # create a sec sliding windows with step 1:
    sw2 = sliding_windows(50, step)
    # add padding on the samples left and right:
    samples = torch.nn.functional.pad(samples.permute(0, 2, 1), (25, 25), "constant", 0).permute(0, 2, 1)
    labels = torch.nn.functional.pad(labels.unsqueeze(1), (25, 25), "reflect").squeeze()
    samples, _ = sw2(samples, None)
    labels = labels.unfold(1, 50, step)
    samples = samples.reshape(-1, 50, 6)
    labels = labels.reshape(-1, 50)
    return samples, labels

def non_fsl_dataloaders_with_sec_sliding(config):
    """
    Get dataloaders for the train, val, test sets for non-FSL

    Args:
        config (_type_): NA
    """
    inputs, labels = _load(config)
    sw = sliding_windows(300, 50)
    segmented_samples, segmented_labels = sw(torch.tensor(inputs), torch.tensor(labels))
    # Split the dataset into train, val and test:
    train_samples, test_samples, train_labels, test_labels = train_test_split(segmented_samples, segmented_labels, test_size=0.2, random_state=42)
    train_samples, val_samples, train_labels, val_labels = train_test_split(train_samples, train_labels, test_size=0.2, random_state=42)

    train_samples, train_labels = sec_sliding(train_samples, train_labels)
    val_samples, val_labels = sec_sliding(val_samples, val_labels)
    test_samples, test_labels = sec_sliding(test_samples, test_labels)
    print(train_samples.shape, val_samples.shape, test_samples.shape, train_labels.shape, val_labels.shape, test_labels.shape)
    train_set = NormalDataset(train_samples, train_labels)
    val_set = NormalDataset(val_samples, val_labels)
    test_set = NormalDataset(test_samples, test_labels)
    train_loader = DataLoader(
        train_set,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=0,
        pin_memory=True,)
    val_loader = DataLoader(
        val_set,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=0,
        pin_memory=True,)
    test_loader = DataLoader(
        test_set,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=0,
        pin_memory=True,)
    return train_loader, val_loader, test_loader

In [23]:
train_loader, val_loader, test_loader = non_fsl_dataloaders_with_sec_sliding(config)
len(train_loader), len(val_loader), len(test_loader)


torch.Size([19397, 50, 6]) torch.Size([4851, 50, 6]) torch.Size([6062, 50, 6]) torch.Size([19397, 50]) torch.Size([4851, 50]) torch.Size([6062, 50])


(152, 38, 48)

In [24]:
from train import get_model

model = get_model(config).float().to(device)
# model = UNet(in_channels=6, out_channels=5).float().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
# print number of trainable parameters:
printc('Number of trainable parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))

criterion = torch.nn.CrossEntropyLoss()

best_loss = math.inf
counter_i = 0
pbar = tqdm(range(config['epochs']), postfix={'loss': 0.0, 'miou': 0.0})
for epoch in pbar:
    model.train()
    for images, labels in train_loader:
        b, h, c = images.shape
        # Forward pass
        images = images.float().to(device)
        labels = labels.to(device)
        outputs = model(images)
        outputs = outputs.permute(0, 2, 1)
        # probs = F.softmax(outputs, dim=1)
        loss= criterion(outputs, labels.long())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1., norm_type=2)
        optimizer.step()
        # scheduler.step(loss)
        # train loss and accuracy check:
        # if counter_i % 1000 == 0 and counter_i != 0:
        #     print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, config['epochs'], loss.item()))
    val_losses = []
    mean_ious = []
    model.eval()
    for images, labels in val_loader:
        images = images.float().to(device)
        labels = labels.to(device)
        outputs = model(images)
        outputs = outputs.permute(0, 2, 1)
        loss = criterion(outputs, labels.long())
        val_losses.append(loss.item())
        mean_ious.append(mean_iou(model.forward_pred(images), labels.long(),num_classes=5))
    val_loss = sum(val_losses) / len(val_losses)
    miou = sum(mean_ious) / len(mean_ious)
    # print(f'Epoch [{epoch+1}/{config["epochs"]}], Val Loss: {val_loss}, Mean IoU: {miou}')
    pbar.set_postfix({'loss': val_loss, 'miou': miou.item()})
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), f'./saved_model/opportunity_{config["model"]}_sec_sliding.pth')
    # visualize the predictions:
    # plt.plot(images[0, :].cpu().detach().numpy(), color='black')
    # plt.plot(model.forward_pred(images)[0, :].cpu().detach().numpy(), label='Prediction')
    # plt.plot(labels[0, :].cpu().detach().numpy(), label='Ground Truth')
    # plt.legend()
    # plt.show()
    counter_i += 1
# Test the model
model.eval()

Number of trainable parameters: 650117


 86%|████████▌ | 172/200 [06:37<01:04,  2.31s/it, loss=0.306, miou=0.876]


KeyboardInterrupt: 